In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = dataset = pd.read_csv('/kaggle/input/reskill-dataset-v1/reskill_dataset_v1.csv')
df.head()

,resume_text,skills
0,david garcia lopez daviddgl@gmail.com https //...,"ionic, flutter, aws, bdd, ionic, flutter, node..."
1,manjunath email manjunathjava261@gmail.com mob...,"java, j2ee, spring, hibernate, jdbc, servlets,..."
2,avinash kumar itpl main road 6th cross kundalh...,"html, html5, css, javascript, jquery, bootstra..."
3,contact ibropamela@gmail.com +44 7383 151 935 ...,"microsoft office, hris, adp, opentable, seven ..."
4,vandana . salesforce consultant profile deadli...,"salesforce, lightning, apex, javascript, visua..."


In [4]:
df.shape

(10325, 2)

In [5]:
# independent features
X = list(df['resume_text'])
y = list(df['skills'])

In [7]:
y[:5]

['ionic, flutter, aws, bdd, ionic, flutter, node.js, docker, kibana, c#, .net, git, mvc, reactjs, angular, javascript, grafana, graphite, sql server, mysql, aurora, postgresql, mongodb',
 'java, j2ee, spring, hibernate, jdbc, servlets, jsp, javascript, html, css, maven, svn, rtc, sql, tomcat, weblogic, websphere, eclipse, windows, ms office, linux, soap, jax-ws, restful, log4j, junit, jquery, microservices, jpa, mysql',
 'html, html5, css, javascript, jquery, bootstrap, asp.net, c#, sql server, 2008, 2008r2, 2012, 2014, 2017, mvc, web api, ssis, ssrs, ajax, linq, vb.net, .net core, angular, entity framework, visual studio, ssms, tfs, git, jira, windows 7, windows 8, windows 10, sap crystal reports',
 'microsoft office, hris, adp, opentable, seven rooms',
 'salesforce, lightning, apex, javascript, visual force, soql, angularjs, html/css, data loader, ant, github, visual studio code, rest api, lwc']

In [8]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [9]:
X_train[6]

'arun kumar mail virath_arun@yahoo.com cell +91-9540265782 objective seeking to serve as a software professional with an esteemed organization which will be provide me the opportunity to learn and grow. career profile 11.3 years of development experience on microsoft technologies. 4+ years of experience of working on sitecore sxa technical skills cms tool microsoft.net rdbms languages tools / packages design patterns additional sitecore 9.3 sxa sitecore 8.2 unicorn vs 2019 2017 mvc 5.2 4.0 sql server 2017 2015 2012 c# jquery visual studio 2008 2010 2012 2015 helix mvc di repository pipe web socket window application work experience working with omnia fz lcc as sitecore developer since december 2021. working with edynamic soft tech pvt. ltd. as senior associate since april 2016 to december 2021. worked with myzeal i.t. solutions llc. as a software developer since june 2014 to april 2016 worked with millennium i-sys pvt. ltd. as a software developer since july 2010 to june 2014. projects

In [10]:
y_train[6]

'c#, .net, asp.net, sql server, mvc, html, javascript, jquery, bootstrap, angularjs, sitecore, unicorn, visual studio, microsoft .net core, mongodb, azuredevops'

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# Tokenizing

X_train_encodings = tokenizer(X_train, truncation = True, padding = True)
X_test_encodings = tokenizer(X_test, truncation = True, padding = True)
y_train_encodings = tokenizer(y_train, truncation = True, padding = True)
y_test_encodings = tokenizer(y_test, truncation = True, padding = True)

In [13]:
# y_train_encodings_2 = [tokenizer.tokenize(skill) for skill in y_train]
# y_test_encodings_2 = [tokenizer.tokenize(skill) for skill in y_test]

# y_train_encodings_2 = torch.tensor([torch.tensor(skill) for skill in y_train_encodings])
# y_test_encodings_2 = torch.tensor([torch.tensor(skill) for skill in y_test_encodings])

In [14]:
# import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_train_encodings),
#     y_train_encodings
# ))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_test_encodings),
#     y_test_encodings
# ))

In [16]:
from torch.utils.data import Dataset

# Custom dataset class
class ResumeSkillsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item, self.labels[idx]

# Create dataset instances
train_dataset = ResumeSkillsDataset(X_train_encodings, y_train_encodings)
test_dataset = ResumeSkillsDataset(X_test_encodings, y_test_encodings)

In [17]:
train_dataset

In [18]:
# batch_size = 16
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
from transformers import BertModel, Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 10,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 16,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
)

In [24]:
# Train
    
trainer = Trainer(
    model = BertModel.from_pretrained('google-bert/bert-base-uncased'),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
)

trainer.train()

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'